# 03. Mock Photon Comparison

In this Notebook we compare the mock photon propagation with the existing methods like the normal flow. To do that, we generate a line detector and a couple of events that we propagate using each propagator.

In [ ]:
import pandas as pd

from ananke.configurations.detector import DetectorConfiguration
from ananke.services.detector import DetectorBuilderService
from olympus.event_generation.generators import get_generator
from olympus.event_generation.medium import MediumEstimationVariant, Medium
from olympus.event_generation.photon_propagation.mock_photons import MockPhotonPropagator
from olympus.event_generation.photon_propagation.norm_flow_photons import NormalFlowPhotonPropagator
from olympus.configuration.generators import GenerationConfiguration
from olympus.configuration.photon_propagation import MockPhotonPropagatorConfiguration, NormalFlowPhotonPropagatorConfiguration

Let's define the detector next:

In [ ]:
oms_per_line = 20
dist_z = 50  # m
dark_noise_rate = 16 * 1e-5  # 1/ns
side_len = 100  # m
pmts_per_module = 16
pmt_cath_area_r = 75e-3 / 2  # m
module_radius = 0.21  # m
efficiency = 0.42 # Christian S. Number

detector_configuration = DetectorConfiguration.parse_obj(
    {
        "string": {
            "module_number": oms_per_line,
            "module_distance": dist_z
        },
        "pmt": {
            "efficiency": efficiency,
            "noise_rate": dark_noise_rate,
            "area": pmt_cath_area_r
        },
        "module": {
            "radius": module_radius
        },
        "geometry": {
            "type": "single",
        },
        "seed": 31338
    }
)

detector_service = DetectorBuilderService()
det = detector_service.get(configuration=detector_configuration)

In [ ]:
det.df[['module_location_x', 'module_location_y', 'module_location_z']].drop_duplicates()

Next up we generate our events:

In [ ]:
configuration = GenerationConfiguration.parse_obj({
    'generator': {
        'type': 'cascade',
        'spectrum': {
            'log_minimal_energy': 2.0,
            'log_maximal_energy': 5.5,
        }
    },
    'number_of_samples': 2
})

cascades_generator = get_generator(
    detector=det,
    configuration=configuration.generator
)

records = cascades_generator.generate_records(
    number_of_samples=2
)

sources = cascades_generator.propagate(records)

records.df.head()

In [ ]:
sources.df.head()

Now we need our photon propagators

In [ ]:
mock_photon_propagator_configuration = MockPhotonPropagatorConfiguration(resolution=18000)

mock_photon_propagator = MockPhotonPropagator(
    detector=det,
    configuration=mock_photon_propagator_configuration
)

normal_photon_propagator_configuration = NormalFlowPhotonPropagatorConfiguration(
    shape_model_path="../../hyperion/data/photon_arrival_time_nflow_params.pickle",
    counts_model_path="../../hyperion/data/photon_arrival_time_counts_params.pickle"
)

normal_photon_propagator = NormalFlowPhotonPropagator(
    detector=det,
    configuration=normal_photon_propagator_configuration
)

Now lets Propagate:

In [ ]:
mock_hits = mock_photon_propagator.propagate(records, sources)

mock_hits.df.head()

And doing the same with the normal photons.

In [ ]:
normal_hits = normal_photon_propagator.propagate(records, sources)

normal_hits.df.head()

## Comparison of the photon propagators

Now that we have all the hits we want we can compare the following cases:

1. number of hits per module
2. hit arrival times


### Number of hits per module

Let's have a look at the mock propagation:

In [ ]:
aggregated_mock_hits = mock_hits.df.set_index(['string_id', 'module_id'])
aggregated_mock_hits.groupby(level=[0,1]).count().head()

In [ ]:
aggregated_normal_hits = normal_hits.df.set_index(['string_id', 'module_id'])
aggregated_normal_hits.groupby(level=[0,1]).count().head()

In [ ]:
len(mock_hits)

In [ ]:
len(normal_hits)